In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os.path


import seaborn as sns
from scipy import stats


# Data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

from matplotlib.pyplot import *
import imageio
from skimage.io import imread

from tqdm import tqdm

import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import NullFormatter
import time
import re
import requests
import pickle
import seaborn as sns
import os
import glob
import sys
#from natsort import natsorted
sns.set()

import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

# Sklearn Liraries
import sklearn
from sklearn import preprocessing

import datetime
from datetime import timedelta, date 
start = time.time()
%matplotlib inline


import sklearn
from sklearn.preprocessing import LabelEncoder    #For encoding categorical variables
from sklearn.model_selection import train_test_split #For splitting of data
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
print(sklearn.__version__)

1.2.1


In [3]:
#!conda install -c conda-forge scikit-learn=0.23.2
#!pip install --pre pycaret

In [4]:
#set directory for csv files
dir_name = r'C:\Users\vasan\OneDrive\EmissionsDatathon\Zeroing Methane Emissions - Dataset'
filename_suffix = 'xls'

In [5]:
skiprows = 0
#Means read in the ',' as thousand seperator. Also drops all columns which are unnamed.
df = pd.read_excel("dfeat3all.xlsx", thousands=',', skiprows = skiprows)
df.head()

,Unnamed: 0,Red,Green,Blue,Name,qplume
0,0,181.358816,47.365528,58.732138,ang20190922t192642-2_r4578_c217-plume.png,2579.528100
1,1,163.016873,116.526079,86.684041,ang20190923t163307-G_r4113_c198-plume.png,934.430300
2,2,148.860910,65.633528,55.371571,ang20190926t223557-D_r4139_c225-plume.png,443.098484
3,3,195.260858,106.511460,97.770766,ang20191006t150453-6_r4168_c237-plume.png,1032.395030
4,4,175.007143,133.739831,96.091491,ang20191007t182352-2_r4062_c150-plume.png,826.823256


In [6]:
df.shape

(6062, 6)

In [7]:
df = df.drop('Unnamed: 0', axis=1)

In [8]:
df = df.drop('Name', axis=1)

In [9]:
df.head()

,Red,Green,Blue,qplume
0,181.358816,47.365528,58.732138,2579.528100
1,163.016873,116.526079,86.684041,934.430300
2,148.860910,65.633528,55.371571,443.098484
3,195.260858,106.511460,97.770766,1032.395030
4,175.007143,133.739831,96.091491,826.823256


In [10]:
df = df.astype(float)

In [11]:
np.isnan(df).any()

Red        True
Green      True
Blue       True
qplume    False
dtype: bool

In [12]:
df.isna().sum()

Red         2
Green      10
Blue      365
qplume      0
dtype: int64

In [13]:
df = df.dropna()

In [14]:
df.shape

(5697, 4)

In [15]:
z_scores = np.abs((df['qplume'] - df['qplume'].mean()) / df['qplume'].std())

In [16]:
df = df[z_scores < 3]

In [17]:
df.shape

(5584, 4)

In [18]:
# Creates a mask where values that are true go into the training/test set
# Note that I done it so that the random number is predictable

msk = np.random.seed(0)
msk = np.random.rand(len(df))<0.8

raw_train_validate_set = df[msk]
raw_test_set = df[~msk]

print(raw_train_validate_set.shape)
print(raw_test_set.shape)

(4480, 4)
(1104, 4)


In [19]:
raw_train_validate_set.to_excel(r'dftrain.xlsx', index = False, header=True)
raw_test_set.to_excel(r'dftest.xlsx', index = False, header=True)

In [20]:
skiprows = 0
#Means read in the ',' as thousand seperator. Also drops all columns which are unnamed.
dftrain = pd.read_excel("dftrain.xlsx", thousands=',', skiprows = skiprows)
dftrain = df.loc[:, ~df.columns.str.contains('^Unnamed')] 
dftrain.head()

,Red,Green,Blue,qplume
0,181.358816,47.365528,58.732138,2579.528100
1,163.016873,116.526079,86.684041,934.430300
2,148.860910,65.633528,55.371571,443.098484
3,195.260858,106.511460,97.770766,1032.395030
4,175.007143,133.739831,96.091491,826.823256


In [21]:
target_columns = [
    "qplume"
]

y_train = dftrain[target_columns]
X_train = dftrain.drop(columns=target_columns)

In [22]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

LinearRegression()

In [23]:
y_predtrain = model_lr.predict(X_train)

In [24]:
print('Coefficients:', model_lr.coef_)
print('Intercept:', model_lr.intercept_)

Coefficients: [[ 0.35329406  0.73865584 -1.28310683]]
Intercept: [470.54809736]


In [25]:
# Calculate the mean squared error
mse = mean_squared_error(y_train, y_predtrain)

# Calculate the root mean squared error
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)

r2 = r2_score(y_train, y_predtrain)
print("Test R^2 Score: {:.5f}".format(r2))

Root Mean Squared Error: 515.5592197017262
Test R^2 Score: 0.00491


In [26]:
skiprows = 0
#Means read in the ',' as thousand seperator. Also drops all columns which are unnamed.
dftest = pd.read_excel("dftest.xlsx", thousands=',', skiprows = skiprows)
dftest = df.loc[:, ~df.columns.str.contains('^Unnamed')] 
dftest.head()

,Red,Green,Blue,qplume
0,181.358816,47.365528,58.732138,2579.528100
1,163.016873,116.526079,86.684041,934.430300
2,148.860910,65.633528,55.371571,443.098484
3,195.260858,106.511460,97.770766,1032.395030
4,175.007143,133.739831,96.091491,826.823256


In [27]:
y_test = dftest[target_columns]
X_test = dftest.drop(columns=target_columns)

In [28]:
y_predtest = model_lr.predict(X_test)

In [29]:
# Calculate the mean squared error
mse = mean_squared_error(y_train, y_predtest)

# Calculate the root mean squared error
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)

r2 = r2_score(y_train, y_predtest)
print("Test R^2 Score: {:.5f}".format(r2))

Root Mean Squared Error: 515.5592197017262
Test R^2 Score: 0.00491


In [30]:
skiprows = 0
#Means read in the ',' as thousand seperator. Also drops all columns which are unnamed.
dsub = pd.read_excel("dsub.xlsx", thousands=',', skiprows = skiprows)
dsub = dsub.loc[:, ~dsub.columns.str.contains('^Unnamed')] 
dsub.head()

,Red,Green,Blue
0,82.213236,86.101068,93.274235
1,185.965074,179.501219,151.516240
2,183.407890,166.772655,131.994160
3,139.278398,132.372221,116.203694
4,110.705144,106.453771,118.587240


In [31]:
dsub.shape

(20, 3)

In [32]:
y_predsub = model_lr.predict(dsub)

In [33]:
y_predsub

array([[443.5117942 ],
       [474.42655544],
       [489.17000208],
       [468.4300883 ],
       [436.13216927],
       [444.6655884 ],
       [460.0764472 ],
       [470.85007359],
       [441.48847548],
       [482.96026758],
       [452.57168207],
       [434.97687922],
       [464.06585353],
       [448.49034524],
       [426.92460084],
       [465.66411153],
       [474.75616608],
       [468.12084103],
       [464.99851756],
       [466.12225983]])

In [31]:
np.savetxt('12.csv', y_predsub, delimiter=',')